In [1]:
from gpt import *
from configs import get_configs
from dataset import StepDPODataset, MetaMathDataset

c:\Users\pc\.conda\envs\hoang\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\pc\.conda\envs\hoang\lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
c:\Users\pc\.conda\envs\hoang\lib\site-packages\torchvision\transforms\v2\__init__

In [2]:
train_ds = MetaMathDataset(block_size=1024,
                                     split='train',
                                     max_examples=None,
                                     tokenizer_name="tiktoken/gpt2")
test_ds = MetaMathDataset(block_size=1024,
                                    split='test',
                                    max_examples=None,
                                    tokenizer_name="tiktoken/gpt2")

Generating train split: 100%|██████████| 395000/395000 [00:02<00:00, 136946.79 examples/s]


Loading MetaMathDataset train split
82717773
Loading MetaMathDataset test split
4378115


In [3]:
print(len(train_ds.pairs))

9174


In [15]:
from tokenizer import tiktoken

In [16]:
enc = tiktoken.get_encoding("gpt2")
encode = lambda s: enc.encode(s, allowed_special={"<|endoftext|>"})

In [17]:
decode = lambda l: enc.decode(l)

In [13]:
print((train_ds.pairs[1078]))

tensor([[ 5756,   338,   892,  ..., 50256, 50256, 50256],
        [ 5756,   338,   892,  ..., 50256, 50256, 50256]])


In [23]:
print(decode(list(train_ds.pairs[1048][0])))

Let's think step by step. 
Step 1: 
Billy has 8 hours of free time on each day of the weekend, so he has a total of 8 x 2 = 16 hours of free time over the weekend.

Step 2: 
He wants to spend 75% of his time playing video games, so he will spend 0.75 x 16 = 12 hours playing video games.

Step 3: 
He spends the remaining time reading books, which is 16 - 12 = 4 hours.<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endof

In [2]:
train_prompt = StepDPOPromptsDataset(block_size=1024,
                                    #  split='train',
                                     max_examples=None,
                                     tokenizer_name="tiktoken/gpt2")
# test_prompt = StepDPOPromptsDataset(block_size=1024,
#                                     split='test',
#                                     max_examples=None,
#                                     tokenizer_name="tiktoken/gpt2")

Generating train split: 100%|██████████| 5398/5398 [00:00<00:00, 899696.13 examples/s]


Loading StepDPOPromptsDataset


In [3]:
print(train_prompt.prompts[0])

[tensor([20490,    25,   314,  ..., 50256, 50256, 50256]), tensor([1, 1, 1,  ..., 0, 0, 0]), tensor(459)]


In [2]:
cfg = get_configs('gpt2-medium')

In [3]:
rm1 = GPTRewardModel.from_checkpoint(cfg, r'runs\rm_hh_rlhf_202411092036_step40199.pt')
rm2 = GPTRewardModel.from_checkpoint(cfg, r'runs\rm_hh_rlhf_202411100543_final.pt')
rm3 = GPTRewardModel.from_checkpoint(cfg, r'runs\rm_hh_rlhf_202411100543_step40199.pt')


In [4]:
from dataset import AnthropicHHRLHFDataset
from torch.utils.data import DataLoader

In [5]:
test_ds = AnthropicHHRLHFDataset(block_size=1024,
                                    split='test',
                                    max_examples=20,
                                    tokenizer_name="tiktoken/gpt2")
test_dataloader = DataLoader(test_ds, batch_size=1, shuffle=True)


Found cached dataset json (C:/Users/pc/.cache/huggingface/datasets/Anthropic___json/Anthropic--hh-rlhf-a9fdd36e8b50b8fa/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
rm1 = rm1.to(device)
# rm2 = rm2.to(device)
# rm3 = rm3.to(device)

In [7]:
import tqdm

In [8]:
all_pos_score=  []
for step, (completions, attention_masks) in enumerate(tqdm.tqdm(
                        test_dataloader)):
                        completions = completions.to(device)
                        attention_masks = attention_masks.to(device)

                        positive_scores = rm1(
                            completions[:, 0, :],
                            attention_masks[:, 0, :]) 
                        # + rm2(completions[:, 0, :],
                            # attention_masks[:, 0, :]) 
                        # + rm3(completions[:, 0, :],attention_masks[:, 0, :])   # (B, 1)
                        
                        all_pos_score.append(positive_scores[0][0])
                        if step == 10:
                                break
                        # neg_scores.extend(negative_scores)
                        

 25%|██▌       | 5/20 [00:14<00:43,  2.93s/it]


KeyboardInterrupt: 